In [2]:
"""
U-Net Prediction Script (Export to multi-page TIFF)

What it does
------------
- Loads a U-Net with your weights.
- Reads a two-channel time-series TIFF (interleaved pages: ch1, ch2, ch1, ch2, ...).
- Reads a GT TIFF (used ONLY to detect fully-black frames and keep index alignment).
- Predicts segmentation for non-black frames, then reinserts black frames as all-zero
  class maps at their original indices.
- Exports a single multi-page TIFF of predictions with values {0, 85, 170, 255}
  corresponding to classes {0, 1, 2, 3}.

Requirements
------------
Python >= 3.9
pip install numpy opencv-python tifffile tqdm tensorflow scipy

Inputs you must set
-------------------
- MODEL_PATH:     path to your trained weights (compatible with the U-Net below).
- DATA_TIF_PATH:  path to the data TIFF (interleaved ch1/ch2 pages).
- LABEL_TIF_PATH: path to the GT TIFF (used only to detect black frames & keep alignment).
- OUTPUT_DIR:     folder to write the predictions TIFF into.

Outputs
-------
- OUTPUT_TIF_PATH = <OUTPUT_DIR>/all_predictions.tif
  A multi-page TIFF with one page per frame (class map encoded as class_index * 85).
  Black frames from the GT are present at their original indices as all-zero pages.
"""

import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPool2D, Conv2DTranspose, Concatenate, BatchNormalization, Activation
)
from tqdm import tqdm

# =========================
# USER CONFIG (EDIT THESE)
# =========================
MODEL_PATH     = "model.h5"                       # path to weights file
DATA_TIF_PATH  = "position.tif"                   # data TIFF (interleaved ch1/ch2)
LABEL_TIF_PATH = "seg.tif"                        # GT TIFF (only for black-frame indices)
OUTPUT_DIR     = "output_predictions"             # output folder for predictions TIFF

# Optional caps / behavior
MAX_FRAMES_TO_EVAL = None   # None to use all frames; otherwise limit to first N frames
TRANSPOSE_FIRST_N  = 1000   # apply cv2.transpose on the first N GT frames (to match your training convention)

# =========================
# DERIVED PATHS (no edit)
# =========================
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_TIF_PATH = os.path.join(OUTPUT_DIR, "all_predictions.tif")

# =============================================================================
# Helpers
# =============================================================================
def pad_image_to_multiple(image: np.ndarray, factor: int = 16):
    """
    Pad spatial dimensions (H, W) up to a multiple of 'factor' using reflection.
    Keeps channel dimension unchanged.

    Args:
        image (np.ndarray): input image of shape (H, W, C).
        factor (int): target multiple for H and W.

    Returns:
        tuple:
            - padded (np.ndarray): padded image (H', W', C).
            - pad_h (int): number of bottom rows added.
            - pad_w (int): number of rightmost cols added.
    """
    h, w, _ = image.shape
    pad_h = (factor - h % factor) if h % factor != 0 else 0
    pad_w = (factor - w % factor) if w % factor != 0 else 0
    padded = np.pad(image, ((0, pad_h), (0, pad_w), (0, 0)), mode="reflect")
    return padded, pad_h, pad_w


def unpad_prediction(pred: np.ndarray, pad_h: int, pad_w: int):
    """
    Remove padding previously added by pad_image_to_multiple.

    Args:
        pred (np.ndarray): prediction of shape (H+pad_h, W+pad_w, num_classes).
        pad_h (int): rows to remove from bottom.
        pad_w (int): cols to remove from right.

    Returns:
        np.ndarray: unpadded prediction of shape (H, W, num_classes).
    """
    if pad_h > 0:
        pred = pred[:-pad_h, :, :]
    if pad_w > 0:
        pred = pred[:, :-pad_w, :]
    return pred


def conv_block(inputs, num_filters: int):
    """
    Two Conv2D -> BN -> ReLU layers with 'same' padding.

    Args:
        inputs: Keras tensor.
        num_filters (int): filters for both conv layers.

    Returns:
        Keras tensor: processed features.
    """
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def encoder_block(inputs, num_filters: int):
    """
    Encoder stage: conv_block + 2x2 MaxPool.

    Args:
        inputs: Keras tensor.
        num_filters (int): filters in conv_block.

    Returns:
        tuple:
            - skip: features (for skip connection).
            - pooled: downsampled features.
    """
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(inputs, skip, num_filters: int):
    """
    Decoder stage: up-convolution -> concatenate skip -> conv_block.

    Args:
        inputs: decoder input tensor.
        skip: encoder skip tensor to concatenate.
        num_filters (int): filters in conv_block.

    Returns:
        Keras tensor: decoded features.
    """
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape):
    """
    Build the U-Net segmentation model used for inference.

    Args:
        input_shape (tuple): e.g. (None, None, 2) for full-frame inference.

    Returns:
        tensorflow.keras.Model: U-Net with softmax over 4 classes.
    """
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b1 = conv_block(p4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    outputs = Conv2D(4, 1, padding="same", activation="softmax")(d4)
    return Model(inputs, outputs)


def load_trained_model(model_path: str):
    """
    Build U-Net and load weights from disk.

    Args:
        model_path (str): path to weights (compatible with this architecture).

    Returns:
        tensorflow.keras.Model: ready for inference.
    """
    print(f"Loading weights from: {model_path}")
    model = build_unet((None, None, 2))
    model.load_weights(model_path)
    print("Weights loaded successfully.")
    return model


def normalize_channel(image: np.ndarray) -> np.ndarray:
    """
    Percentile normalization after CLAHE+blur (applied earlier in the pipeline).
    If dynamic range is degenerate, returns zeros.

    Args:
        image (np.ndarray): 2D array (uint8/float).

    Returns:
        np.ndarray: float32 image in [0, 1].
    """
    per99 = np.percentile(image, 99)
    per1 = np.percentile(image, 1)
    eps = 1e-6
    if per99 - per1 < eps:
        return np.zeros_like(image, dtype=np.float32)
    image = np.clip(image, per1, per99)
    return ((image - per1) / (per99 - per1)).astype(np.float32)


def read_segmentation_frames(tiff_path: str, transpose_first_n: int = TRANSPOSE_FIRST_N, max_frames=None):
    """
    Read GT TIFF pages (odd indices), detect fully-black frames, and apply 'undefined->3'.

    NOTE: The returned GT is not used for metrics here; only black indices are used to
    skip prediction and to reinsert black pages at the end.

    Args:
        tiff_path (str): path to GT TIFF.
        transpose_first_n (int): apply cv2.transpose to the first N frames.
        max_frames (int|None): cap the number of frames to read.

    Returns:
        tuple:
            - y_true (np.ndarray): GT class maps (0..3) for non-black frames (kept for shape consistency).
            - black_indices (list[int]): indices of fully-black frames.
    """
    import tifffile as tiff
    from scipy.ndimage import binary_dilation

    def separate_undefined_regions(frame: np.ndarray) -> np.ndarray:
        """
        Mark 'undefined' background regions (0) far from cells (1/2) as class 3.
        """
        frame = frame.copy()
        zero_mask = (frame == 0)
        cell_mask = (frame == 1) | (frame == 2)
        dilated_cells = binary_dilation(cell_mask)
        undefined_mask = zero_mask & (~dilated_cells)
        frame[undefined_mask] = 3
        return frame.astype(np.uint8)

    y_true = []
    black_indices = []
    frame_counter = 0

    with tiff.TiffFile(tiff_path) as tif:
        all_indices = list(range(1, len(tif.pages), 2))  # GT pages at odd indices
        if max_frames is not None:
            all_indices = all_indices[:max_frames]

        for _, i in tqdm(enumerate(all_indices), total=len(all_indices), desc="Reading GT (black-skip only)"):
            frame = tif.pages[i].asarray()
            idx = frame_counter
            if np.all(frame == 0):
                black_indices.append(idx)
                frame_counter += 1
                continue
            if idx < transpose_first_n:
                frame = cv2.transpose(frame)
            y_true.append(separate_undefined_regions(frame))
            frame_counter += 1

    return np.array(y_true), black_indices


def read_data_frames(tiff_path: str, skip_indices=None, max_frames=None):
    """
    Read data TIFF frames as interleaved pairs (ch1, ch2), apply CLAHE+blur+normalization,
    and stack into (H, W, 2). Skips frames whose indices appear in 'skip_indices'.

    Args:
        tiff_path (str): path to data TIFF (interleaved channels).
        skip_indices (Iterable[int]|None): frame indices to skip (e.g., black frames from GT).
        max_frames (int|None): cap the number of frames to read.

    Returns:
        np.ndarray: array of shape (N_non_skipped, H, W, 2), dtype float32.
    """
    if skip_indices is None:
        skip_indices = []

    import tifffile as tiff

    X_data = []
    frame_counter = 0
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    with tiff.TiffFile(tiff_path) as tif:
        total_frames = len(tif.pages) // 2
        num_frames = total_frames if max_frames is None else min(max_frames, total_frames)

        for i in tqdm(range(num_frames), desc="Reading data"):
            if frame_counter in skip_indices:
                frame_counter += 1
                continue

            ch1 = tif.pages[2 * i].asarray()
            ch2 = tif.pages[2 * i + 1].asarray()

            ch1 = clahe.apply(ch1)
            ch2 = clahe.apply(ch2)
            ch1 = cv2.GaussianBlur(ch1, (3, 3), 0)
            ch2 = cv2.GaussianBlur(ch2, (3, 3), 0)

            ch1 = normalize_channel(ch1)
            ch2 = normalize_channel(ch2)

            X_data.append(np.stack([ch1, ch2], axis=-1).astype(np.float32))
            frame_counter += 1

    return np.array(X_data)


# =============================================================================
# Prediction + export with black-frame reinsertion
# =============================================================================
def run_prediction_and_save_tif():
    """
    Full pipeline:
        1) Load model and weights.
        2) Read GT to collect black frame indices (and preserve alignment).
        3) Read data frames, skipping black indices.
        4) Predict per frame with padding-to-multiple-of-16 and unpadding.
        5) Reinsert black frames as all-zero class maps at original indices.
        6) Write a multi-page TIFF to OUTPUT_TIF_PATH with values {0,85,170,255}.
    """
    import tifffile as tiff

    model = load_trained_model(MODEL_PATH)
    _, black_indices = read_segmentation_frames(
        LABEL_TIF_PATH, transpose_first_n=TRANSPOSE_FIRST_N, max_frames=MAX_FRAMES_TO_EVAL
    )
    X_data = read_data_frames(
        DATA_TIF_PATH, skip_indices=black_indices, max_frames=MAX_FRAMES_TO_EVAL
    )

    print(f"\nPredicting {len(X_data)} non-black frames...")
    y_pred_classes = []

    for i in tqdm(range(len(X_data)), desc="Predicting"):
        img = X_data[i]
        padded, pad_h, pad_w = pad_image_to_multiple(img)
        pred = model.predict(np.expand_dims(padded, axis=0), verbose=0)[0]
        unpadded = unpad_prediction(pred, pad_h, pad_w)
        y_pred_classes.append(np.argmax(unpadded, axis=-1).astype(np.uint8))

    # Reinsert black frames at original indices as all-zero class maps
    num_black = len(black_indices)
    total_frames = num_black + len(y_pred_classes)
    black_set = set(black_indices)

    # Determine output frame shape
    if len(y_pred_classes) > 0:
        H, W = y_pred_classes[0].shape
    else:
        # Fallback: read dimensions from the first odd GT page
        import tifffile as tiff  # local import to match original pattern
        with tiff.TiffFile(LABEL_TIF_PATH) as tif_gt:
            sample = tif_gt.pages[1].asarray()
            H, W = sample.shape

    assembled = []
    pred_ptr = 0
    for idx in range(total_frames):
        if idx in black_set:
            assembled.append(np.zeros((H, W), dtype=np.uint8))
        else:
            assembled.append(y_pred_classes[pred_ptr])
            pred_ptr += 1

    # Encode classes as values {0,85,170,255} and save
    stack = (np.stack(assembled, axis=0).astype(np.uint8) * 85).astype(np.uint8)  # (N, H, W)
    tiff.imwrite(OUTPUT_TIF_PATH, stack)
    print(f"\nSaved multi-page TIFF to: {OUTPUT_TIF_PATH}")


if __name__ == "__main__":
    run_prediction_and_save_tif()


Loading weights from: /content/drive/MyDrive/Final_Project/experiments/exp_19_Full_Data_set/model_epoch_10.h5
✅ Weights loaded successfully.


Reading data: 100%|██████████| 50/50 [00:17<00:00,  2.84it/s]



🔍 Predicting 48 frames (non-black only)...


Predicting: 100%|██████████| 48/48 [14:52<00:00, 18.60s/it]



✅ Saved multi-page TIF (with black frames reinserted) to: /content/drive/MyDrive/Final_Project/output_predictions/all_predictions.tif
